In [1]:
!pip install scikit-learn==1.0.2
!pip install eli5

# Ignore future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Imports
import os
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import eli5
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
from IPython.display import display_html
from sklearn.metrics import confusion_matrix

# Constants
PATH_TO_DATA = r'/kaggle/input/open-ml-course-linear-models-spring22/'
SEED = 241

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/26.5 MB ? eta -:--:--

     ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/26.5 MB 19.8 MB/s eta 0:00:02

     ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/26.5 MB 24.5 MB/s eta 0:00:01

     ━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/26.5 MB 37.7 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/26.5 MB 48.1 MB/s eta 0:00:01

...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 971.9 kB/s eta 0:00:00


  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2


    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2



[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


E0000 00:00:1752496609.660728      74 common_lib.cc:612] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:230


In [2]:
def prepare_sparse_features(path_to_train, path_to_test, path_to_site_dict, vectorizer_params):
    # Define time columns
    times = ['time%s' % i for i in range(1, 11)]
    
    # Load datasets
    train_df = pd.read_csv(path_to_train, index_col='session_id', parse_dates=times)
    test_df = pd.read_csv(path_to_test, index_col='session_id', parse_dates=times)
    
    # Sort by session start time
    train_df = train_df.sort_values(by='time1')
    
    # Load site dictionary
    with open(path_to_site_dict, 'rb') as f:
        site2id = pickle.load(f)
    id2site = {v: k for k, v in site2id.items()}
    id2site[0] = 'unknown'  # id 0 means unknown
    
    # Extract site columns
    sites = ['site%s' % i for i in range(1, 11)]
    
    # Convert site IDs to names
    train_sessions = train_df[sites].fillna(0).astype('int').apply(
        lambda row: ' '.join([id2site[i] for i in row]), axis=1).tolist()
    test_sessions = test_df[sites].fillna(0).astype('int').apply(
        lambda row: ' '.join([id2site[i] for i in row]), axis=1).tolist()
    
    # Vectorize sessions using TF-IDF
    vectorizer = TfidfVectorizer(**vectorizer_params)
    X_train = vectorizer.fit_transform(train_sessions)
    X_test = vectorizer.transform(test_sessions)
    y_train = train_df['target'].astype('int').values
    
    return X_train, X_test, y_train, vectorizer, train_df[times], test_df[times]

In [3]:
# Load data and prepare features
X_train_sites, X_test_sites, y_train, vectorizer, train_times, test_times = prepare_sparse_features(
    path_to_train=os.path.join(PATH_TO_DATA, 'train.csv'),
    path_to_test=os.path.join(PATH_TO_DATA, 'test.csv'),
    path_to_site_dict=os.path.join(PATH_TO_DATA, 'site_dic.pkl'),
    vectorizer_params={'ngram_range': (1, 3),
                       'max_features': 29000, 
                       'tokenizer': lambda s: s.split()}
)

/usr/local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [4]:
# Load site dictionary and reverse it
sites_dict = pd.read_pickle(os.path.join(PATH_TO_DATA, 'site_dic.pkl'))
sites_dict_inv = {v: k for k, v in sites_dict.items()}

# Read raw training and test sets
sites_train = ['site%s' % i for i in range(1, 11)] + ['target']
sites_test = ['site%s' % i for i in range(1, 11)]

train_sites = pd.read_csv(os.path.join(PATH_TO_DATA, 'train.csv'),
                       index_col='session_id', parse_dates=['time%s' % i for i in range(1, 11)])
test_sites = pd.read_csv(os.path.join(PATH_TO_DATA, 'test.csv'),
                       index_col='session_id', parse_dates=['time%s' % i for i in range(1, 11)])

# Clean up
train_sites = train_sites.sort_values(by='time1')
train_sites = train_sites[sites_train].fillna(0).astype('int')
test_sites = test_sites[sites_test].fillna(0).astype('int')

In [5]:
features_name = []
corr_df = []
time_split = TimeSeriesSplit(n_splits=10)

# Save predictions to submission file
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    predicted_df = pd.DataFrame(predicted_labels,
                                index=np.arange(1, predicted_labels.shape[0] + 1),
                                columns=[target])
    predicted_df.to_csv(out_file, index_label=index_label)

In [6]:
def train_and_predict(model, X_train, y_train, X_test,
                      site_feature_names=None,
                      new_feature_names=None, 
                      cv=time_split, scoring='roc_auc',
                      top_n_features_to_show=30, 
                      submission_file_name='submission.csv'):
    """
    Train a model, evaluate with cross-validation, interpret feature weights, 
    and generate predictions for submission.

    Parameters:
    - model: sklearn estimator (e.g., LogisticRegression)
    - X_train: training feature matrix (sparse)
    - y_train: training labels
    - X_test: test feature matrix (sparse)
    - site_feature_names: list of site feature names (from TF-IDF vectorizer)
    - new_feature_names: list of additional engineered feature names
    - cv: cross-validation strategy (e.g., TimeSeriesSplit)
    - scoring: evaluation metric (default 'roc_auc')
    - top_n_features_to_show: number of top features to display via eli5
    - submission_file_name: output file for test predictions
    """

    # Evaluate model using cross-validation
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring=scoring, n_jobs=4)
    print('CV scores:', cv_scores)
    print('CV mean:', cv_scores.mean(), ', CV std:', cv_scores.std())

    # Fit the model on the full training set
    model.fit(X_train, y_train)

    # Auto-detect site feature names if not provided
    if site_feature_names is None:
        try:
            site_feature_names = vectorizer.get_feature_names_out()
        except AttributeError:
            site_feature_names = vectorizer.get_feature_names()

    # Combine TF-IDF and custom feature names
    all_feature_names = list(site_feature_names)
    if new_feature_names:
        all_feature_names += new_feature_names

    # Display top features using ELI5
    display_html(eli5.show_weights(estimator=model, 
                                   feature_names=all_feature_names,
                                   top=top_n_features_to_show))

    # Optionally print weights of additional features
    if new_feature_names:
        print('New feature weights:')
        print(pd.DataFrame({
            'feature': new_feature_names,
            'coef': model.coef_.flatten()[-len(new_feature_names):]
        }))

    # Evaluate on training data
    proba = model.predict_proba(X_train)
    predicted = model.predict(X_train)
    table_confusion = confusion_matrix(y_train, predicted)

    # Predict probabilities on the test set
    test_pred = model.predict_proba(X_test)[:, 1]

    # Write predictions to CSV
    write_to_submission_file(test_pred, submission_file_name)

    return (proba, y_train, predicted, table_confusion, cv_scores)

In [7]:
# Time features from session start
def add_time_features(times, X_sparse, add_feat=True):
    hour = times['time1'].apply(lambda t: 100 * t.hour + t.minute) / 1000

    # Define segments of day
    morning_1 = (((hour >= 0.901) & (hour <= 0.904) | (hour >= 0.922) & (hour <= 1.209)).astype('int') * hour).values.reshape(-1, 1)
    morning_2 = (((hour >= 0.905) & (hour <= 0.921)).astype('int') * hour).values.reshape(-1, 1)
    day_1 = (((hour >= 1.210) & (hour <= 1.239)).astype('int') * hour).values.reshape(-1, 1)
    day_2 = (((hour >= 1.240) & (hour <= 1.335)).astype('int') * hour).values.reshape(-1, 1)
    day_3 = (((hour >= 1.336) & (hour <= 1.358)).astype('int') * hour).values.reshape(-1, 1)
    day_4 = (((hour >= 1.359) & (hour <= 1.517)).astype('int') * hour).values.reshape(-1, 1)
    day_5 = (((hour >= 1.518) & (hour <= 1.553)).astype('int') * hour).values.reshape(-1, 1)
    evening_1 = (((hour >= 1.554) & (hour <= 1.629) | (hour >= 1.705) & (hour <= 1.755)) * hour).values.reshape(-1, 1)
    evening_2 = ((hour >= 1.653) & (hour <= 1.704)).values.reshape(-1, 1)
    evening_3 = (((hour >= 1.756) & (hour <= 1.828) | (hour >= 1.626) & (hour <= 1.656)) * hour).values.reshape(-1, 1)
    night = (((hour >= 1.829) & (hour <= 2.359) | (hour >= 0) & (hour <= 0.900)) * hour).values.reshape(-1, 1)
    
    # Combine all
    objects_to_hstack = [X_sparse, morning_1, morning_2, day_1, day_2, day_3, day_4, day_5,
                         evening_1, evening_2, evening_3, night]
    feature_names = ['morning_1', 'morning_2', 'day_1', 'day_2', 'day_3', 'day_4',
                     'day_5', 'evening_1', 'evening_2', 'evening_3', 'night']
    
    if add_feat:
        for i, j in zip(objects_to_hstack[1:], feature_names):
            corr_df.append(pd.DataFrame(i, columns=[j]))
    
    return hstack(objects_to_hstack), feature_names

In [8]:
# Add day of the week features
def add_day_month(times, X_sparse, add_feat=True):
    day_of_week = times['time1'].apply(lambda t: t.weekday())
    day_of_week_df = pd.get_dummies(day_of_week)
    day_of_week_df['5_6'] = day_of_week_df[5] + day_of_week_df[6]
    day_of_week_df['2_3'] = day_of_week_df[2] + day_of_week_df[3]
    for d in (2, 3, 5, 6):
        del day_of_week_df[d]
    day_of_week_df = day_of_week_df.rename({i: f'weekday_{i}' for i in day_of_week_df.columns}, axis=1)
    
    if add_feat:
        corr_df.append(day_of_week_df.reset_index(drop=True))
    
    return hstack([X_sparse, day_of_week_df]), list(day_of_week_df.columns)

# Add day of month features
def add_dom(times, X_sparse, add_feat=True):
    dom = times['time1'].apply(lambda ts: ts.day)
    dom_1 = (dom.isin([3,5,6,7,8,10,11,12,21,23,27,28,30])).values.reshape(-1, 1)
    dom_2 = (dom.isin([9,24])).values.reshape(-1, 1)
    dom_3 = (dom.isin([17,18,19,20,21,22,24,25,26,31])).values.reshape(-1, 1)
    
    if add_feat:
        corr_df.extend([
            pd.DataFrame(dom_1, columns=['dom_1']),
            pd.DataFrame(dom_2, columns=['dom_2']),
            pd.DataFrame(dom_3, columns=['dom_3']),
        ])
        
    return hstack([X_sparse, dom_1, dom_2, dom_3]), ['dom_1', 'dom_2', 'dom_3']

In [9]:
def train_and_predict(model, X_train, y_train, X_test,
                      site_feature_names=None,
                      new_feature_names=None,
                      cv=None,
                      scoring='roc_auc',
                      top_n_features_to_show=30,
                      submission_file_name='submission.csv'):
    import numpy as np
    from IPython.display import display_html

    # Combine feature names carefully
    if site_feature_names is None:
        site_feature_names = []
    if new_feature_names is None:
        new_feature_names = []

    all_feature_names = list(site_feature_names) + list(new_feature_names)

    # Check length consistency
    if X_train.shape[1] != len(all_feature_names):
        raise ValueError(f"Feature name length mismatch! "
                         f"X_train has {X_train.shape[1]} features, "
                         f"but combined feature_names has {len(all_feature_names)}.")

    # Cross-validation scores
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring=scoring, n_jobs=4)
    print('CV scores:', cv_scores)
    print('CV mean:', cv_scores.mean(), ', CV std:', cv_scores.std())

    # Fit model
    model.fit(X_train, y_train)

    # Show feature weights with eli5
    import eli5
    display_html(eli5.show_weights(estimator=model, feature_names=all_feature_names,
                                   top=top_n_features_to_show))

    # Optionally print weights of new features separately (if any)
    if new_feature_names:
        print('New feature weights:')
        # Assuming logistic regression with coef_ shape (1, n_features)
        coef = model.coef_.flatten()
        new_feats_coef = coef[-len(new_feature_names):]
        print(pd.DataFrame({'feature': new_feature_names, 'coef': new_feats_coef}))

    # Evaluate model on train set
    proba = model.predict_proba(X_train)[:, 1]
    predicted = model.predict(X_train)
    table_confusion = confusion_matrix(y_train, predicted)
    test_pred = model.predict_proba(X_test)[:, 1]

    # Save submission file (make sure this function exists in your code)
    write_to_submission_file(test_pred, submission_file_name)

    return proba, y_train, predicted, table_confusion, cv_scores


# ---- Updated training code example ----

# After all feature engineering steps, you should get the final feature matrix and features_name list:

# Example (you should adjust these based on your actual feature engineering functions)
X_train_final, new_feat_names = add_time_features(train_times, X_train_sites)
X_train_final, more_feat_names = add_day_month(train_times, X_train_final)
X_train_final, dom_features = add_dom(train_times, X_train_final)

# Combine all new feature names
features_name = new_feat_names + more_feat_names + dom_features

# Do the same for test data but without adding features (add_feat=False)
X_test_final, _ = add_time_features(test_times, X_test_sites, add_feat=False)
X_test_final, _ = add_day_month(test_times, X_test_final, add_feat=False)
X_test_final, _ = add_dom(test_times, X_test_final, add_feat=False)

# IMPORTANT: Check the shape and feature names length BEFORE training
print("X_train_final shape:", X_train_final.shape)
print("Length of vectorizer feature names:", len(vectorizer.get_feature_names_out()))
print("Length of new features:", len(features_name))
print("Total expected features:", len(vectorizer.get_feature_names_out()) + len(features_name))

# Make sure X_train_final has the expected number of features
assert X_train_final.shape[1] == len(vectorizer.get_feature_names_out()) + len(features_name), \
    "Feature matrix and feature names length mismatch!"

final_model = LogisticRegression(C=20, random_state=SEED, solver='liblinear')
proba, ideal, predicted, confusion_mat, cv_scores = train_and_predict(
    model=final_model,
    X_train=X_train_final,
    y_train=y_train,
    X_test=X_test_final,
    site_feature_names=vectorizer.get_feature_names_out(),
    new_feature_names=features_name,
    cv=time_split,
    submission_file_name='submission.csv'
)

X_train_final shape: (253561, 29019)


Length of vectorizer feature names: 29000
Length of new features: 19
Total expected features: 29019


CV scores: [0.83240866 0.9750089  0.97879763 0.87582835 0.96021256 0.93738
 0.89697062 0.96465718 0.98907643 0.98360991]
CV mean: 0.9393950224824286 , CV std: 0.050602718711326114


Weight?,Feature
+18.688,cid-ed6c3e6a5c6608a4.users.storage.live.com
+11.441,video.tt
+11.378,www.banque-chalus.fr
+10.433,t.voyages-sncf.com
+10.216,static.weezbe.com
+10.168,www.video.tt
+9.870,www.qapa.fr
+9.353,zupimages.net
+9.091,www.tete-en-lair.com
+8.996,www.thefreecamsecret.com


New feature weights:


        feature      coef
0     morning_1 -4.300382
1     morning_2 -1.558681
2         day_1 -0.746022
3         day_2 -0.233323
4         day_3 -1.169838
5         day_4 -2.820155
6         day_5 -4.448594
7     evening_1  0.332500
8     evening_2 -0.601184
9     evening_3  0.621469
10        night -8.634162
11    weekday_0  0.904057
12    weekday_1  0.677531
13    weekday_4  0.021906
14  weekday_5_6 -5.708145
15  weekday_2_3 -1.778256
16        dom_1 -3.209101
17        dom_2  1.053452
18        dom_3 -0.970429
